In [1]:
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time 
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import configparser

In [3]:
import time

In [4]:
user_id = "hctmkr" 
user_pass = "hcinc3600"

In [5]:
!pwd

/Users/jayz/Autoreport


In [6]:
# options = webdriver.ChromeOptions()
# driver = webdriver.Chrome('./chromedriver')
# driver = webdriver.Chrome('./User/desktop/documents/notebook/DSS_17/chrome/chromedriver')
# driver = webdriver.Chrome()
driver = webdriver.Chrome('/Users/jayz/dayz/chrome/chromedriver')

In [7]:
#driver.set_window_size(1400, 1200)
driver.get("https://www.kaida.co.kr/uat/uia/egovLoginUsr.do")

In [8]:
driver.find_element_by_css_selector('#userId').send_keys(user_id)
driver.find_element_by_css_selector('#userPass').send_keys(user_pass)
driver.execute_script('actionLogin()')

In [9]:
# 2. 사용자 설정통계 페이지 이동

In [10]:
driver.get("https://www.kaida.co.kr/ko/statistics/custom2.do")

In [11]:
# 3. 브랜드, 지역, 구매유형, 연령 데이터 수집

In [12]:
# 구매유형
selector = '#pageBody > article.body_wrap > div > div.select_filter > table > tbody > tr > td:nth-child(8) > div > label'
sales_types = driver.find_elements_by_css_selector(selector)
sales_types = {sales_type.text: sales_type.get_attribute("for") for sales_type in sales_types}
print(len(sales_types), sales_types)

4 {'개인': 'PurchaseType3', '개인-남자': 'PurchaseType1', '개인-여자': 'PurchaseType2', '법인': 'PurchaseType0'}


In [13]:
# 연령
ages = driver.find_elements_by_css_selector(".td_Age > .item_wrap > label")
ages = {age.text: age.get_attribute("for") for age in ages}
print(len(ages), ages)

7 {'~19': 'Age1', '20~29': 'Age2', '30~39': 'Age3', '40~49': 'Age4', '50~59': 'Age5', '60~69': 'Age6', '70~': 'Age7'}


In [14]:
def clear_menu(driver):
    # 나이 체크박스 초기화
    driver.execute_script("$('#td_Age > .item_wrap > label > input').prop('checked', false);")
    # tag 초기화
    driver.execute_script("$('.sel_item button').click();");

In [15]:
def select_menu(driver, sales_type_id, age_id):
    
    # 메뉴 초기화
    clear_menu(driver)
    time.sleep(0.5)
    
    # 구매유형 클릭
    driver.find_element_by_css_selector(f'input#{sales_type_id}').click()
    time.sleep(0.5)
    
    # 연령대 클릭
    try: 
        driver.find_element_by_css_selector(f'input#{age_id}').click()
        time.sleep(0.5)
    except:
        pass

In [16]:
def make_table_df(driver, sales_type, age, year):
    
    # 컬럼 데이터 수집
    columns = driver.find_elements_by_css_selector("table.re_table > thead > tr > th")
    columns = [column.text for column in columns] + [ "sales_type", "age", "year"] 

    # 데이터 프레임 만들기
    df = pd.DataFrame(columns=columns)

    # 데이터 프레임 채우기
    rows = driver.find_elements_by_css_selector("table.re_table > tbody > tr")
    for row in rows:
        cells = row.find_elements_by_css_selector('td,th')
        datas = [cell.text for cell in cells] + [sales_type, age, year]     
        df.loc[len(df)] = datas
    
    return df

In [17]:
sales_type

NameError: name 'sales_type' is not defined

In [17]:
dfs = []
for year in range(2020, 2022): #2016, 2022        
    years = Select(driver.find_element_by_xpath("""//*[@id="searchYear"]"""))
    driver.find_element_by_css_selector('#searchDivision > option:nth-child(2)').click()

    for sales_type, sales_type_id in list(sales_types.items()):
        for age, age_id in list(ages.items()):
            print([sales_type, age, year], end=" ")

                    # 메뉴 선택
            select_menu(driver, sales_type_id, age_id)

                    # 검색버튼 클릭
            driver.find_element_by_css_selector('.searchBtn').click()
            time.sleep(0.5)

                    # 데이터 수집해서 데이터 프레임으로 만들기
            try:
                df = make_table_df(driver, sales_type, age, year)
                dfs.append(df)
            except:
                pass

                
# 수집된 데이터 프레임 병합
result_df = pd.concat(dfs)

# index 초기화
result_df.reset_index(drop=True, inplace=True)

# csv 파일로 저장
result_df.to_csv("result1.csv", index=False, encoding='utf-8-sig')
result_df.to_excel('result1.xlsx', index=False)

# 출력
result_df.tail()

['개인', '~19', 2020] ['개인', '20~29', 2020] ['개인', '30~39', 2020] ['개인', '40~49', 2020] ['개인', '50~59', 2020] ['개인', '60~69', 2020] ['개인', '70~', 2020] ['개인-남자', '~19', 2020] ['개인-남자', '20~29', 2020] ['개인-남자', '30~39', 2020] ['개인-남자', '40~49', 2020] ['개인-남자', '50~59', 2020] ['개인-남자', '60~69', 2020] ['개인-남자', '70~', 2020] ['개인-여자', '~19', 2020] ['개인-여자', '20~29', 2020] ['개인-여자', '30~39', 2020] ['개인-여자', '40~49', 2020] ['개인-여자', '50~59', 2020] ['개인-여자', '60~69', 2020] ['개인-여자', '70~', 2020] ['법인', '~19', 2020] ['법인', '20~29', 2020] ['법인', '30~39', 2020] ['법인', '40~49', 2020] ['법인', '50~59', 2020] ['법인', '60~69', 2020] ['법인', '70~', 2020] ['개인', '~19', 2021] ['개인', '20~29', 2021] ['개인', '30~39', 2021] ['개인', '40~49', 2021] ['개인', '50~59', 2021] ['개인', '60~69', 2021] ['개인', '70~', 2021] ['개인-남자', '~19', 2021] ['개인-남자', '20~29', 2021] ['개인-남자', '30~39', 2021] ['개인-남자', '40~49', 2021] ['개인-남자', '50~59', 2021] ['개인-남자', '60~69', 2021] ['개인-남자', '70~', 2021] ['개인-여자', '~19', 2021] ['개인-여자', '20~

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year
15299,Volvo - XC60 B5 AWD,0,0,0,28,0,0,0,0,0,0,0,0,28,법인,70~,2021
15300,Volvo - XC60 B6 AWD,0,0,9,16,0,0,0,0,0,0,0,0,25,법인,70~,2021
15301,Volvo - XC60 T8 AWD,78,112,45,16,0,0,0,0,0,0,0,0,251,법인,70~,2021
15302,Volvo - XC90 B6 AWD,0,0,20,22,0,0,0,0,0,0,0,0,42,법인,70~,2021
15303,Volvo - XC90 T8 AWD,49,45,43,27,0,0,0,0,0,0,0,0,164,법인,70~,2021


In [18]:
import re

In [19]:
 d = [column for column in result_df["구분"]]

In [20]:
re.findall('(\w+)\s-', str(d))

['Audi',
 'BMW',
 'BMW',
 'Bentley',
 'Honda',
 'Rover',
 'MINI',
 'Maserati',
 'Benz',
 'Benz',
 'Porsche',
 'Volkswagen',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'Audi',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'BMW',
 'Cadillac',
 'Cadillac',
 'Cadillac',
 'Chevrolet',
 'Chevrolet',
 'Chevrolet',
 'Chevrolet',
 'Chevrolet',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Chrysler',
 'Ci

In [21]:
re.findall('-\s(\w+\s?\w+\s?\w+\s+\w+)', str(d))

['Q2 35 TDI',
 '218d Gran Coupe',
 'Continental GT V8',
 'Defender 110 D240',
 'MINI Cooper five',
 'E 350 4MATIC',
 'Cayenne Turbo Coupe',
 'Tiguan Allspace 2',
 'A4 40 TDI quattro',
 'A4 40 TFSI',
 'A4 45 TFSI quattro',
 'A5 Cabriolet 45 TFSI',
 'A5 Coupe 45 TFSI',
 'A5 Sportback 40 TDI',
 'A5 Sportback 40 TFSI',
 'A5 Sportback 45 TFSI',
 'A6 40 TDI',
 'A6 40 TDI quattro',
 'A6 45 TDI quattro',
 'A6 45 TFSI',
 'A6 45 TFSI quattro',
 'A6 50 TDI quattro',
 'A7 45 TDI quattro',
 'A7 55 TFSI quattro',
 'A8 L 50 TDI',
 'Q2 35 TDI',
 'Q5 40 TDI quattro',
 'Q7 55 TFSI quattro',
 'Q8 45 TDI quattro',
 'Q8 50 TDI quattro',
 'Q8 55 TFSI quattro',
 'SQ5 TDI',
 '118d Urban',
 '218d Gran Coupe',
 '320 Touring',
 '320d Touring',
 '320d xDrive',
 '420 Coupe',
 '420d Coupe',
 '523d xDrive',
 '530 xDrive',
 '620d Gran Turismo',
 '620d xDrive Gran Turismo',
 '730Ld xDrive',
 '740Li xDrive',
 '840 xDrive Gran Coupe',
 'Active Tourer',
 'M235 xDrive Gran Coupe',
 'M340 xDrive Touring',
 'M440 xDrive Cou

In [22]:
driver.quit()

In [23]:
result_df

,구분,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Total,sales_type,age,year
0,Audi - Q2 35 TDI,1,0,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2020
1,BMW - 218d Gran Coupe,0,1,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2020
2,BMW - 520,1,0,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2020
3,Bentley - Continental GT V8,0,1,0,0,0,0,0,0,0,0,0,0,1,개인,~19,2020
4,Honda - CR-V Hybrid,0,0,0,1,0,0,0,0,0,0,0,0,1,개인,~19,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15299,Volvo - XC60 B5 AWD,0,0,0,28,0,0,0,0,0,0,0,0,28,법인,70~,2021
15300,Volvo - XC60 B6 AWD,0,0,9,16,0,0,0,0,0,0,0,0,25,법인,70~,2021
15301,Volvo - XC60 T8 AWD,78,112,45,16,0,0,0,0,0,0,0,0,251,법인,70~,2021
15302,Volvo - XC90 B6 AWD,0,0,20,22,0,0,0,0,0,0,0,0,42,법인,70~,2021


In [24]:
result_df.to_csv("result1.csv", index=False, encoding='utf-8-sig')